### Load libraries

In [14]:
import yaml
import json
import requests
from datetime import datetime,timedelta
import time
import math
from google.cloud import storage

### Configuration from a YAML file

In [15]:
config_file = open('parameter_request.yaml', 'r')
config = yaml.safe_load(config_file)

### Enviroment variables

In [16]:
bucket = config['bucket']
base_url = 'https://api.coingecko.com/api/v3/'
crypto=config["cryptos"]
date_start=config["period"][0]
date_end=config["period"][1]

days=(datetime.strptime(date_end, '%d-%m-%Y').date()-datetime.strptime(date_start, '%d-%m-%Y').date()).days
dates = [(datetime.strptime(date_start, '%d-%m-%Y').date()+timedelta(days=x)).strftime("%d-%m-%Y") for x in range(days+1)]

### Search id from symbol

To do this, we make a Request that list all coins with id, name, and symbol

In [17]:
request=base_url+"coins/list"
cat=requests.get(request).json()
crypto_id=[]
for i in crypto:
    crypto_id.append(next(item for item in cat if item["symbol"] == i)["id"])    

### API GET Request

We set a rate limit of 35 calls by minute, this way API don't block us

In [18]:
data=[]
calls=len(crypto_id)*len(dates)
rate_minute_call=35

i=0
for cry in crypto_id:
    for dt in dates:
        i=i+1
        if (i % rate_minute_call)==1:
            ahora=datetime.today()
        if (i % rate_minute_call)==0:
            segundos=60-(datetime.today()-ahora).seconds
            time.sleep(segundos) 
        request_body=base_url+f'coins/{cry}/history?date={dt}&localization=false'
        payload=requests.get(request_body).json()
        payload['date'] = dt
        payload['crypto'] = cry
        data.append(payload)     

### Write into Google Cloud Storage

In [ ]:
client = storage.Client()
gcs_bucket = client.get_bucket(bucket)

In [ ]:
for row in data:
    path = f"crypto/{row['symbol']}/data_{row['date']}.json"
    blob = gcs_bucket.blob(path)
    with blob.open(mode = 'w') as file:
        json.dump(row, file)